In [1]:
%%time

import pandas as pd
import numpy as np
import nltk
from collections import defaultdict

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

Wall time: 2.76 s


#### 1. Getting data:

In [2]:
def get_data(url):    
    df = pd.read_csv(url, parse_dates=['date'])
    df = df[~df.headline.isna()] # Cleaning
    
    df = df[['headline', 'category']]
    
    return df

In [ ]:
file_name = 'News_Category_Dataset.csv'

df = get_data(file_name)

features = df.headline
labels = df.category

features_split = [nltk.word_tokenize(sentence.lower()) for sentence in features]

# Pipeline

In [3]:
file_name = 'News_Category_Dataset.csv'

df = get_data(file_name)

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['headline'], df['category'], test_size=.1, random_state=1)

In [6]:
from sklearn.base import BaseEstimator
from collections import defaultdict
import numpy as np

class CustomPreProcess(BaseEstimator):
    
    def default_val(self): # Pickle can't serialize lambda
        return 0
    
    def __init__(self, keep_nbr=200, debug=False):
        self.keep_nbr = keep_nbr
        self.word_to_idx = defaultdict(self.default_val)
        self.num_cols = 0 # When searching for non existing index, it adds it to word_to_idx => after transform len increases
        self.debug = debug
        pass
    
    def tokenizing(self, X):
        return [nltk.word_tokenize(sentence.lower()) for sentence in X]
    
    def make_dictionnary(self, rows):
        # Getting word frequency
        word_freq = {}
        for sentence in rows:
            for word in sentence:
                if word not in word_freq:
                    word_freq[word] = 1
                else:
                    word_freq[word] += 1
                    
        # Reducing dictionnary size based on word frequency
        word_freq = dict(sorted(word_freq.items(), key=lambda x: x[1], reverse=True))
        words = [word for word, freq in word_freq.items() if freq > self.keep_nbr]
        
        # Creating dictionnary
        idx = 1
        for word in words:
            self.word_to_idx[word] = idx
            idx += 1
            
        self.num_cols = len(self.word_to_idx) + 1
    
    def fit(self, X, y=None): 
        if self.debug: print('Tokenizing... Might take some time')
        split_X = self.tokenizing(X)
        if self.debug: print('Making dictionnary')
        self.make_dictionnary(split_X)
        if self.debug: print('Done!')
        pass
    
    def transform(self, X):
        if self.debug: print('Tokenizing... Might take some time')
        split_X = self.tokenizing(X)
        
        if self.debug: print('Transforming to vec')
        num_samples = X.shape[0]

        features_vec = np.zeros((num_samples, self.num_cols), dtype=np.int8)
        for sample_idx in range(num_samples):
            for word in split_X[sample_idx]:
                word_idx = self.word_to_idx[word]
                features_vec[sample_idx][word_idx] = 1
        
        return features_vec
    
    def fit_transform(self, X, y=None):
        self.fit(X, y),
        return self.transform(X)

In [6]:
%%time
pre_proc = CustomPreProcess()
pre_proc.fit(X_train, y_train)
X_train_preproc = pre_proc.transform(X_train)
X_test_preproc = pre_proc.transform(X_test)

NameError: name 'default_val' is not defined

In [7]:
print(X_train.shape)
print(X_test.shape)
print(X_train_preproc.shape)
print(X_test_preproc.shape)

(180762,)
(20085,)
(180762, 1269)
(20085, 1269)


In [8]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth=5)
tree.fit(X_train_preproc, y_train)
preds_test = tree.predict(X_test_preproc)

In [20]:
total_accuracy(preds_test, y_test)

22.389843166542196

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

model_pipeline = Pipeline(steps=[
    ('pre_processing', CustomPreProcess()),
    ('DecisionTreeClassifier', DecisionTreeClassifier(max_depth=5))
])

In [7]:
model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('pre_processing', CustomPreProcess()),
                ('DecisionTreeClassifier',
                 DecisionTreeClassifier(max_depth=5))])

In [17]:
preds_test = model_pipeline.predict(X_test)

In [18]:
preds_test

array(['POLITICS', 'POLITICS', 'POLITICS', ..., 'POLITICS', 'POLITICS',
       'POLITICS'], dtype=object)

In [8]:
import joblib

joblib.dump(model_pipeline, 'First_quick_model.joblib')

['First_quick_model.joblib']

In [7]:
import joblib

model = joblib.load('First_quick_model.joblib')

In [9]:
import numpy as np

title = np.array(['this is a test'])

In [19]:
[w for w in title]

['this is a test']

In [11]:
model.predict(title)[0]

'POLITICS'

# Other

#### 2. Word frequency

In [4]:
def get_word_freq(rows, sort=True):
    word_freq = {}
    for sentence in rows:
        for word in sentence:
            if word not in word_freq:
                word_freq[word] = 1
            else:
                word_freq[word] += 1
                
    if sort:
        word_freq = dict(sorted(word_freq.items(), key=lambda x: x[1], reverse=True))
        
    return word_freq

In [5]:
word_freq = get_word_freq(features_split)

In [6]:
%%time

min_f = 200

print(f'{len(word_freq)} words in corpus')
words = [word for word, freq in word_freq.items() if freq > min_f]
percentage = len(words)/len(word_freq)*100
print(f'Reduced to {len(words)} keeping those with freq > {min_f} ({percentage:.2f} %)')

71565 words in corpus
Reduced to 1412 keeping those with freq > 200 (1.97 %)
Wall time: 13 ms


In [7]:
%%time

word_to_idx = defaultdict(lambda:0)
idx = 1
for word in words:
    word_to_idx[word] = idx
    idx += 1

Wall time: 1e+03 µs


In [8]:
num_samples = features.shape[0]
num_cols = len(word_to_idx) + 1

features_vec = np.zeros((num_samples, num_cols), dtype=np.int8)
for sample_idx in range(num_samples):
    for word in features_split[sample_idx]:
        word_idx = word_to_idx[word]
        features_vec[sample_idx][word_idx] = 1

In [4]:
import time

print(f'time {time.time():.2f}s')

time 1611332387.85s


#### Train / Test Split

In [9]:
%%time

X_train, X_test, y_train, y_test = train_test_split(features_vec, labels, test_size=.1, random_state=1)

Wall time: 182 ms


#### Training

In [10]:
%%time

tree = DecisionTreeClassifier(max_depth=5)
tree.fit(X_train, y_train)

Wall time: 38.4 s


DecisionTreeClassifier(max_depth=5)

#### Evaluation

In [11]:
preds_test = tree.predict(X_test)

In [21]:
tree.predict_proba(X_test)[0]

array([0.00858057, 0.00551975, 0.02121288, 0.03580499, 0.00687767,
       0.01909159, 0.01965678, 0.00456554, 0.01288187, 0.00629046,
       0.05685638, 0.00659875, 0.00871269, 0.03561415, 0.00825761,
       0.015194  , 0.04103114, 0.01758687, 0.0212349 , 0.00537295,
       0.01211116, 0.01053304, 0.04990531, 0.02279834, 0.13814061,
       0.03160645, 0.01445265, 0.01162671, 0.02739324, 0.01333695,
       0.03173124, 0.01292591, 0.01184691, 0.02035409, 0.05171098,
       0.01166341, 0.01450403, 0.11043174, 0.01868054, 0.0118983 ,
       0.01540686])

In [22]:
preds_test[0]

'POLITICS'

In [19]:
def total_accuracy(y_pred, y_true):
    return sum(y_pred == y_true) / len(y_true) * 100

def accuracy_by_group(y_pred, y_true):
    results = {val: 0 for val in set(y_true.values)}
    for i in range(len(y_pred)):        
        label = y_pred[i]

        if y_pred[i] == y_true.values[i]:
            results[label] += 1

    for val, nbr in y_true.value_counts().reset_index().values:
        results[val] /= nbr
        
    results = dict(sorted(results.items(), key=lambda x: x[1], reverse=True))
    
    return results

In [13]:
total_accuracy(preds_test, y_test)

22.389843166542196

- 22.36992780682101
- 22.389843166542196

In [15]:
accuracy_by_group(preds_test, y_test)

{'POLITICS': 0.9171419993995797,
 'STYLE & BEAUTY': 0.4224049331963001,
 'ENTERTAINMENT': 0.4083898827884022,
 'WEDDINGS': 0.34065934065934067,
 'DIVORCE': 0.27627627627627627,
 'ENVIRONMENT': 0.11347517730496454,
 'HOME & LIVING': 0.08823529411764706,
 'COMEDY': 0.07283464566929133,
 'FOOD & DRINK': 0.05605786618444846,
 'QUEER VOICES': 0.05,
 'WORLDPOST': 0.0,
 'FIFTY': 0.0,
 'COLLEGE': 0.0,
 'PARENTS': 0.0,
 'TECH': 0.0,
 'WEIRD NEWS': 0.0,
 'TRAVEL': 0.0,
 'SCIENCE': 0.0,
 'WOMEN': 0.0,
 'GOOD NEWS': 0.0,
 'MEDIA': 0.0,
 'HEALTHY LIVING': 0.0,
 'ARTS & CULTURE': 0.0,
 'WORLD NEWS': 0.0,
 'ARTS': 0.0,
 'BLACK VOICES': 0.0,
 'GREEN': 0.0,
 'TASTE': 0.0,
 'MONEY': 0.0,
 'CRIME': 0.0,
 'BUSINESS': 0.0,
 'PARENTING': 0.0,
 'THE WORLDPOST': 0.0,
 'STYLE': 0.0,
 'CULTURE & ARTS': 0.0,
 'IMPACT': 0.0,
 'LATINO VOICES': 0.0,
 'WELLNESS': 0.0,
 'SPORTS': 0.0,
 'RELIGION': 0.0,
 'EDUCATION': 0.0}

In [16]:
y_train.value_counts()

POLITICS          29407
WELLNESS          16081
ENTERTAINMENT     14437
TRAVEL             8929
STYLE & BEAUTY     8676
PARENTING          7812
HEALTHY LIVING     6027
FOOD & DRINK       5673
QUEER VOICES       5653
BUSINESS           5329
COMEDY             4667
SPORTS             4400
BLACK VOICES       4061
HOME & LIVING      3787
PARENTS            3578
WEDDINGS           3287
THE WORLDPOST      3252
WOMEN              3155
IMPACT             3109
DIVORCE            3093
CRIME              3083
MEDIA              2501
WEIRD NEWS         2400
GREEN              2327
WORLDPOST          2317
RELIGION           2307
STYLE              2048
SCIENCE            1978
WORLD NEWS         1957
TASTE              1883
TECH               1879
MONEY              1529
ARTS               1357
GOOD NEWS          1260
FIFTY              1255
ENVIRONMENT        1182
ARTS & CULTURE     1178
COLLEGE            1037
LATINO VOICES      1014
CULTURE & ARTS      950
EDUCATION           907
Name: category, 

# Tests

In [17]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
for word in sorted(set([word.lower() for sentence in features_split for word in sentence])):
    print(word)

!
#
$
%
&
'
''
''green
''the
'00s
'07
'08
'10
'10,000
'10,000-year
'100
'101
'11
'11/22/63
'12
'13
'133
'13th
'14
'16
'18
'19
'1929
'1970s
'1984
'1989
'20
'20-armed
'20-somethings
'20/20
'2011
'2013
'2015
'2016
'2024
'20s
'20th
'21
'22
'24/7
'24k
'25
'250
'28
'30
'300
'30s-era
'32
'33
'350
'360
'365
'39
'3d
'40
'40s
'42
'46664
'4d-printed
'50
'50s
'545-day
'57
'60
'60s
'60s-style
'61
'63
'64
'650
'650-pound
'67
'70
'70s
'70s-era
'70s-inspired
'72
'73
'74
'7th
'80
'80s
'80s-inspired
'80s-themed
'89
'90
'900
'90210
'90s
'90s-heavy
'90s-inspired
'99
'_____
'aaa
'aahs
'aaron
'aawful
'ab
'abandoned
'abc
'able
'able-bodied
'abnormal
'abolish
'aborted
'abortion
'about
'above
'absences
'absolute
'absolutely
'abstract
'absurd
'abundance
'abuse
'abuse-deterrent
'abuser
'abusers
'abusive
'accelerate
'accept
'acceptable
'acceptance
'access
'accessibility
'accident
'accidental
'accidentally
'accomplishment
'accomplishments
'accountability
'accountable
'accurate
'accused
'achievable
'acoustic
'acqua

'felt
'female
'female-focused
'females
'feminazi
'feminine
'femininity
'feminism
'feminist
'feminists
'fences
'feral
'ferdinand
'ferguson
'ferret
'ferris
'fervent
'festivus
'feta
'fetal
'fetch
'fetish
'fetus
'feud
'fiance
'fiction
'fiddler
'fields
'fierce
'fifth
'fifty
'fight
'fighting
'filibuster
'filthiest
'filthy
'final
'finally
'financial
'financially
'find
'finding
'finds
'fine
'fine-tuned
'finesse
'finishing
'fire
'fireable
'firebaked
'fireball
'firebombed
'fired
'firefighter
'firefly
'firelight
'firenados
'fires
'firewall
'firework
'fireworks
'firing
'firm
'first
'first-rate
'firsthand
'firsts
'fiscal
'fish
'fistful
'fit
'fitness
'five
'five-deferment
'five-year
'fivehead
'fix
'fixer
'fixes
'fizzle
'flabbergasted
'flagship
'flamboyant
'flame
'flammable
'flanders
'flappy
'flash
'flashdance
'flat
'flat-chested
'flattered
'flava
'flavors
'flawed
'flawless
'flaws
'fleabag
'fleur
'flexible
'flight
'flimflam
'flint
'flintstones
'flip
'flippable
'flipping
'flippy
'flirt
'floating
'floc

'pig
'pigeonhole
'pigs
'pile
'pill
'pillar
'pillars
'pillow
'pimp
'pimping
'pina
'pinches
'pink
'pioneer
'pipe
'pipeline
'pippa
'pirate
'pirates
'piss
'pissed
'pissing
'pit
'pitch
'pitchfork
'pitchforks
'pities
'pitzer
'pivot
'pivoted
'pixels
'pixie
'pizza
'pizzagate
'places
'plague
'plain
'plainly
'planes
'planet
'planetarium
'planned
'planning
'plant
'plantation
'plastic
'play
'playbook
'playboy
'played
'playing
'playtime
'pleads
'please
'pleasure
'pledge
'plenty
'plogging
'plugged
'plugged-in
'plus-size
'plush-ass
'pob
'pocahontas
'podium
'poets
'point
'poison
'poisoned
'poisoning
'pokemon
'pokemonument
'pokémon
'polarizing
'police
'policy
'politely
'political
'politically
'politician
'politico
'politics
'poll
'pollen
'polls
'polluter-friendly
'pony
'ponystep
'pooch
'poop
'poopgangsta
'poor
'poorest
'poorly
'pop
'pop-tart
'pope
'porcupine
'porn
'pornographic
'pornography
'portion
'portlandia
'portrait
'portraits
'pose
'posed
'position
'positive
'possessed
'possible
'post
'post-admin

'zombieland
'zombies
'zoo
'zoolander
'zootopia
'zz
(
)
*
+
+e
,
-
--
-do
-family
-garcia
-ing
-la
-s
-stress
-the
-themed
-work
-y
-–
.
..
...
....
.....
.111
.45
/
//
//www.eurweb.com/2015/07/wendy-williwendy
//www.eurweb.com/2015/07/wggendy-williams-announces-12-city-sit-down-tour/
/loses
0
0-3
0.00
0.00064
0.01
0.075
0.5k
0.7
0/hour
00.00
002
007
00s
04/29/12
06290-177
1
1,000
1,000+
1,000,000
1,000-a-head
1,000-day
1,000-percent
1,000-year-old
1,000th
1,001
1,039-person
1,052
1,069
1,083,333
1,100
1,140
1,200
1,200-mile
1,237
1,250
1,290
1,300
1,300-pound
1,325
1,368-pound
1,400
1,410
1,420
1,425
1,432
1,433-pound
1,440
1,500
1,500-gallon
1,500-pound
1,500m
1,558.95
1,600
1,600-bottle
1,750
1,800
1,835,000
1,844
1,900
1-0
1-0.
1-10
1-13
1-2
1-2-3
1-3
1-4
1-5
1-6
1-7
1-7,1914
1-8
1-800
1-hour
1-minute
1-month-old
1-on-1
1-percent
1-percenter
1-second
1-sentence
1-shot
1-year-old
1.019
1.1
1.15
1.2
1.2-acre
1.21
1.23
1.25
1.27
1.3
1.35
1.38
1.3k
1.4
1.42
1.45
1.49
1.5
1.50
1.57
1.5m


acd/labs
ace
aced
aceh
aces
acetaminophen
acevedo
achane
achatz
aches
achievable
achieve
achieved
achievement
achievements
achievers
achieves
achieving
achievment
achilles
aching
achy
acid
acid-wash
acidic
acidification
acids
acing
ackbar
acker
ackerman
ackley
ackman
acknowledge
acknowledged
acknowledgement
acknowledges
acknowledging
acknowledgment
acl
aclidinium
aclj
aclu
acm
acms
acne
acne-prone
acoa
acoas
acord
acorn
acorns
acosta
acoustic
acoustics
acqua
acquaintance
acquaintances
acquainted
acquaints
acquarium
acquire
acquired
acquirer
acquires
acquiring
acquisition
acquit
acquits
acquittal
acquitted
acre
acreage
acres
acrimony
acrobat
acrobatic
acrobatics
acronym
acronyms
across
acroyoga
acs
act
acta
acted
acting
action
action-comedy
action-movie
action-packed
actionable
actionmoviecampaignslogan
actions
activate
activates
activating
activation
active
active-duty
actively
activewear
activiaism
activism
activist
activists
activities
activity
acton
actor
actor-playwright
actor-sing

...

skid
skidmore
skids
skier
skiers
skies
skiier
skiing
skill
skilled
skillet
skillets
skillfully
skillingsbolle
skills
skills-based
skim
skimming
skimp
skimping
skimpy
skin
skin-cancer
skin-care
skin-loving
skin-tight
skincare
skincare-obsessed
skinhead
skinheads
skinless
skinned
skinnier
skinnies
skinniest
skinny
skinny-dipping
skinny-fat
skinnygirl
skins
skintern
skintight
skip
skipped
skipper
skipping
skippy
skips
skirmish
skirt
skirts
skis
skit
skits
skittish
skittles
sklar
skoll
skort
skoza
skrein
skrillex
skripal
skriver
skulking
skull
skullcandy
skullcap
skulls
skunk
skunks
sky
sky-high
skydive
skydiver
skydivers
skydiving
skye
skyfall
skyhouse
skyla
skylar
skyler
skylight
skyline
skylines
skymall
skynyrd
skype
skypes
skyping
skyrobics
skyrocket
skyrocketed
skyrocketing
skyrockets
skyscanner
skyscraper
skyscrapers
skywalker
skywalking
skywatchers
skywriting
slab
slabs
slack
slacker
slacking
slacktivism
slactivism
slade
slager
slags
slain
slainte
slalom
slam
slam-dunk
slam-dunking


subsidizing
subsidy
substance
substances
substandard
substantial
substantially
substitute
substitutes
substitutions
subterranean
subtext
subtitles
subtle
subtleties
subtlety
subtly
subtract
subtracting
subtraction
subtweet
suburb
suburban
suburbia
suburbs
subversion
subversive
subversively
subvert
subverts
subway
subways
subzero
succeed
succeeding
succeeds
success
success-filled
success-seeking
successes
successful
successfully
succession
successor
succulent
succulents
succumb
succumbs
such
suck
suck-up
suck-ups
sucked
sucker
sucker-punched
sucker-punches
suckers
suckiness
sucking
suckla
sucks
sucky
sucralose
suction
suction-cup
sudan
sudanese
sudden
suddenly
sudeikis
sudhof
suds
sue
sued
suede
sues
suey
suffer
suffered
sufferer
sufferers
suffering
suffers
sufficient
sufficiently
suffocate
suffocated
suffocates
suffrage
suffragettes
suffragist
sufi
sugandh
sugar
sugar-coated
sugar-covered
sugar-free
sugar-packed
sugar-spun
sugar-sweetened
sugarcoat
sugarcoating
sugarplum
sugarplums
sug

to-go
toad
toady
toan
toast
toasted
toaster
toasters
toastie
toasting
toasts
toasty
toastymug
tobacco
tobacco-cancer
tobacco-related
tobacco-selling
toback
tobago
tobe
tobey
tobi
tobias
tobin
toblerone
tobruk
toby
tock
tod
today
todays
todd
toddies
toddler
toddler-like
toddlerhood
toddlers
toddlewood
toddy
todos
todrick
toe
toe-slimming
toe-to-toe
toeing
toes
toffee
toffler
tofu
together
togetherness
toggle
togo
toh
tohoku
toil
toilet
toiletries
toiletry
toilets
toiling
toimmigrantswithlove
token
tokens
toking
tokyo
tol
tolan
tolbert
told
toldot
toledo
toler
tolerance
tolerant
tolerate
tolerated
tolerating
tolimán
tolkien
toll
tolle
tolley
tolls
tolstoy
toluca
tom
tomas
tomasi-hill
tomatillo
tomato
tomato-mozzarella
tomato-rich
tomatoes
tomb
tomball
tomboy
tombs
tombstone
tombstones
tomcat
tomei
tomeoni
tomer
tomi
tomlin
tomlinson
tommaso
tommy
tomorrow
tomorrowland
tomorrows
tomorrowworld
tompkins
toms
tomás
tomás-keegan
ton
tondo
tone
tone-deaf
tone-down
toned
toned-down
toner
tones


vegweek
vehement
vehicle
vehicles
vehicular
veil
veiled
veiling
veils
vein
vein-shaped
veins
vekol
velasquez
velcro
velez
velez-mitchell
velghe
velocity
velodrome
veloster
velour
velveeta
velvet
velvety
velvin
velázquez
vena
vendetta
vending
vending-machine
vendor
vendors
vendy
venegas
venerable
veneta
venetian
veneto
venezia
venezuela
venezuelan
venezuelans
vengeance
vengeful
venice
venices
venissimo
venmo
venn
venom
venomous
vent
venter
ventiera
ventimiglia
venting
ventricular
ventriloquist
ventriloquists
vents
ventura
venture
venturebeat
ventured
ventures
venturini
venue
venues
venus
venus-like
veon
vera
veracode
veracruz
veranda
verastegui
verb
verbal
verbally
verdant
verde
verdi
verdict
verdugo
verdun
vereen
vergara
verge
vergne
verhoeven
verifiably
verification
verified
verify
verily
verismo
veritable
verità
verizon
verizon-aol
verizon-contracted
verizon-nj
verizon-wired
verlander
verma
vermeer
vermeulen
vermilion
vermillion
vermont
vermouth
vernacular
vernazza
verne
vernon
verno

ziauddin
zidarevich
zidisha
ziegler
ziel
zielinski
ziering
ziettlow
ziff
zig
zig-zag
zig-zagging
zig-zags
ziggy
zihuacan
zihuatanejo
zihuatenejo
zika
zika-infected
zika-linked
zika-related
zika-transmitting
zilla
zillion
zillow
zimbabwe
zimbabwean
zimbabweans
zimmer
zimmerman
zimmermann
zimmermans
zimmern
zina
zinc
zine
zinepak
zinfandel
zing
zinger
zingers
zinging
zings
zinke
zinzanni
zion
zionism
zionist
zip
zipcar
zipcode
zipline
ziplining
ziploc
zipper
zippered
zippers
zipping
zippori
zirilli
zirx
zisser
zit
zit-zapping
zithromax
ziti
zits
znga
zodiac
zodiacal
zoe
zoellick
zoetrope
zoey
zohydro
zoku
zola
zolciak
zoloft
zolpidem
zoltar
zombicon
zombie
zombie-like
zombies
zone
zones
zong
zonta
zoo
zoobean
zoodles
zooey
zookeeper
zookeepers
zoolander
zoom
zooming
zoonoses
zoonosis
zoos
zoran
zorek
zorro
zosia
zottman
zs
zsa
zsigmond
zubairy
zubaydah
zubchevich
zucchini
zuch
zucker
zuckerberg
zuckerberg-controlled
zuckerman
zuill
zukav
zukerman
zulily
zulu
zuma
zumba
zuni
zuniga
zurich

Number of unique words:
- No modification: 74319
- Lower case: 71552
- 

In [19]:
stopwords.words("english")

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each